In [1]:
import dask.array as da
import numpy as np

from lmdec.decomp.utils import make_snp_array
from lmdec.array.io import save_array, load_array_from_disk

# Creating fake SNP Data

In [2]:
def random_array_with_nan(size= (10000,8000), max_value=3, density=.001):
    min_value = 0
    array = max_value*da.random.random(size=size)
    
    threshold = max_value*(1-density)

    
    array[array > threshold] = float('nan')
    return array

def float_to_int_array(array):
    return da.trunc(array)

## Visualization Example

The following is a toy example to show that a *snp-like* array is generated.

*snp-like* refers to an array drawn from \{0, 1, 2, NaN} where NaN values have density specified

In [3]:
array = random_array_with_nan((10, 10), density=.25)
array = float_to_int_array(array)
array.compute()

array([[ 2.,  1.,  0.,  1.,  1.,  2.,  0.,  0.,  0.,  0.],
       [ 2.,  1., nan,  1., nan,  1.,  0., nan,  1.,  1.],
       [nan,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  2.,  1.,  1.,  0.,  1.,  2.],
       [ 0.,  2.,  0., nan, nan,  0., nan,  0.,  1., nan],
       [ 0., nan,  1.,  0.,  0.,  1.,  2.,  1., nan,  1.],
       [ 0., nan,  0.,  2., nan,  1.,  0., nan,  0.,  0.],
       [nan,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  2.,  2.],
       [ 1.,  1.,  1.,  2., nan,  1.,  1., nan,  1.,  0.],
       [ 1.,  0.,  0.,  1.,  1., nan,  2.,  1.,  1., nan]])

# Example

Creating a large array that has NaNs

In [4]:
N, P = 20000, 10000
density = .00001
print(f'There should be {N*P*density} NaN values')
array = random_array_with_nan((N, P), density=density)
array = float_to_int_array(array)
array

There should be 2000.0000000000002 NaN values


dask.array<trunc, shape=(20000, 10000), dtype=float64, chunksize=(4000, 2500), chunktype=numpy.ndarray>

Create a scaled and centered array

In [5]:
snp_array = make_snp_array(array, mean=True, std=True, std_method='norm')

Columns have 0 mean

In [6]:
snp_array[:, 0:10].mean(axis=0).compute()

array([-5.92415006e-17,  7.30171479e-16, -6.29718500e-17,  7.73603404e-16,
       -7.14095449e-17, -4.42668124e-16, -5.92947913e-16, -3.53672647e-16,
        4.90274488e-16,  1.41753276e-16])

Columns have unit STD

In [7]:
snp_array[:, 0:10].std(axis=0).compute()

array([1.      , 0.99995 , 1.      , 1.      , 1.      , 0.999975,
       0.999975, 1.      , 1.      , 1.      ])

# Memory Usage

`snp_array` is stored in a very structured format which is optimized for matrix multiplication and memory cost.

Essentially, `snp_array` is stored as a "list" of the following arrays:
1. SNP - An array of size (N, P) with elements consisting only of {0, 1, 2} stored as 1 byte integers. All NaN values are replaced with 0.
2. MASK - A sparse array of size (N, P) that has the same shape as SNP, but only has values where the original array had NaN values. Therefore, MASK can hold the non-integer imputed values efficiently and allow SNP to store the rest of the data efficiently.
3. U - An array of size (P,) (The mean of each column, computed from the original array ignoring the NaN values).
4. D - An array of size (P,) (The std of each column, computed from the original array ignoring the NaN values).

Then `snp_array` refers to 

$$(SNP + MASK - U)Inv(D)$$

**NOTE** However, this expression which defined `snp_array` is never executed during matrix multiplication, and should never be executed unless absolutely necessary! It will be tremendously slow and costly.

## Examine the structure of $(SNP + MASK - U)Inv(D)$

If we look just at the expressing:
$$(SNP + MASK)$$
When computed, an array of floats would be created. All NaN values from the original array were replaced with 0 in $SNP$ and a value at the corresponding location in $MASK$ was imputed as the mean of each column. Therefore, $(SNP + MASK)$ will have all non NaN values from the original array and any NaN values replaced with their imputed values resulting in the imputed matrix.

If we look at:
$$((SNP + MASK) - U)$$
We have the imputed matrix on the left in $(SNP + MASK)$. This imputed matrix is then centered by $U$ which will subtract the mean from each column.

If we look at the full array:
$$(SNP + MASK - U)Inv(D)$$
We have the imputed then 0-centered matrix on the left in $(SNP + MASK - U)$. This 0-centered imputed matrix is scaled to have a unit STD for each column.

The memory cost of storing these separate arrays is almost entirely dominated by storing $SNP$, which can now be stored as an array of {0, 1, 2} values instead of an array of floats (NaN is a float). 

In addition, preforming a matrix multiplication with array, `x` is still incredibly fast due to the structure.
$z = (SNP + MASK - U)Inv(D)x$ <br>
$y = Inv(D)x$ <br>
$z = SNPy + MASKy - Uy$




# To confirm this structure lets view the underlying arrays of `snp_array`

If we look at the underlying arrays we see that `snp_array` is a Chained Array with two sub arrays

In [27]:
print(type(snp_array)) 
print(len(list(snp_array.arrays))) # Look at the sub arrays that make up `snp_array`

<class 'lmdec.array.chained.ChainedArray'>
2


Looking at the last item, we see it is a (P, ) array that refers to D, or the column scaling

In [62]:
list(snp_array.arrays)[1]

dask.array<truediv, shape=(10000,), dtype=float64, chunksize=(2500,), chunktype=numpy.ndarray>

Looking at the first item, we see it is a (N, P) StackedArray that refers to $(SNP + MASK - U)$, or the centered masked array.

In [32]:
snp_mask_u = list(snp_array.arrays)[0]
print(type(snp_mask_u))
print(list(snp_mask_u.arrays))

<class 'lmdec.array.stacked.StackedArray'>
[StackedArray(dask.array<rechunk-merge, shape=(20000, 10000), dtype=int8, chunksize=(12000, 7500), chunktype=numpy.ndarray>
             dask.array<array, shape=(20000, 10000), dtype=float64, chunksize=(20000, 10000), chunktype=sparse.COO>), dask.array<neg, shape=(10000,), dtype=float64, chunksize=(2500,), chunktype=numpy.ndarray>]


`snp_mask_u` is a StackedArray of two sub-arrays and not three as it is treated like $((SNP + MASK) - U)$. Where $(SNP + MASK)$ is its own StackedArray. Therefore, if we look at the last sub_array of `snp_mask_u`, we should find $U$.

In [63]:
u = list(snp_mask_u.arrays)[1]
print(type(u))
u

<class 'dask.array.core.Array'>


dask.array<neg, shape=(10000,), dtype=float64, chunksize=(2500,), chunktype=numpy.ndarray>

Therefore, if we look at the first sub_array of `snp_mask_u`, we should find $(SNP + MASK)$.

In [36]:
snp_mask = list(snp_mask_u.arrays)[0]
print(type(snp_mask))
print(list(snp_mask.arrays))

<class 'lmdec.array.stacked.StackedArray'>
[dask.array<rechunk-merge, shape=(20000, 10000), dtype=int8, chunksize=(12000, 7500), chunktype=numpy.ndarray>, dask.array<array, shape=(20000, 10000), dtype=float64, chunksize=(20000, 10000), chunktype=sparse.COO>]


Splitting up `snp_mask` into $SNP$ and $MASK$ is the last set

In [37]:
snp, mask = list(snp_mask.arrays)

Notice that `snp` is a 'int8' array and thus is stored very efficiently. While `mask` is a 'float64' but its chunk type is sparse.COO.

In [38]:
snp

dask.array<rechunk-merge, shape=(20000, 10000), dtype=int8, chunksize=(12000, 7500), chunktype=numpy.ndarray>

In [39]:
mask

dask.array<array, shape=(20000, 10000), dtype=float64, chunksize=(20000, 10000), chunktype=sparse.COO>

Therefore if we compute mask, we can see its memory usage. It is only 50Kb

In [40]:
mask.compute()

Format,coo
Data Type,float64
Shape,"(20000, 10000)"
nnz,2072
Density,1.036e-05
Read-only,True
Size,48.6K
Storage ratio,0.0


## Compare it to a Densely Masked Array

We can replace all NaN values of Array with 0 and add a fill_value. However, this limits us to a constant fill value 

In [50]:
ma_array = da.ma.masked_invalid(array)
da.ma.set_fill_value(ma_array, 1.2324)
ma_array

dask.array<_set_fill_value, shape=(20000, 10000), dtype=float64, chunksize=(4000, 2500), chunktype=numpy.MaskedArray>

If we compute a section of `ma_array` we can see how it is stored.

In [51]:
ma_array[0:5, 0:5].compute()

masked_array(
  data=[[2.0, 1.0, 2.0, 0.0, 2.0],
        [2.0, 1.0, 1.0, 2.0, 0.0],
        [0.0, 0.0, 1.0, 2.0, 0.0],
        [1.0, 0.0, 2.0, 1.0, 2.0],
        [1.0, 0.0, 1.0, 1.0, 1.0]],
  mask=[[False, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False]],
  fill_value=1.2324)

The data is stored as a dense array similar to `snp_array`. However, the mask is a dense array as well that is quite expensive to store.

# Check MatrixMultiplication speed
(We persist both arrays to have a fair comparison)

In [8]:
snp_array = snp_array.persist()

In [52]:
ma_array = ma_array.persist()

In [53]:
y = np.random.randn(P, 2)

In [54]:
%timeit snp_array.dot(y).compute()

394 ms ± 14.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
%timeit ma_array.dot(y).compute()

497 ms ± 8.17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The `snp_array` taking less that twice the time as array is extremely impressive when you consider that:
1. `snp_array` is stored as 1 byte integers and a sparse mask while `array` is stored as a masked floating array. That means `snp_array` takes less than 1/8th the memory as `array`.
2. `snp_array` is also centered and scaled (0 mean and unit std) while `array` does not have these properties.

# Save and re-Load Array
With large arrays where running `make_snp_array` is expensive, or the original `array` is to large to hold in memory with `array.persist()`. We can save the array to disk, which will create `snp_array` in sections, and  then reload it in the proper format. This is beneficial because:
1. We can save the 1 byte SNP section of `snp_array` in a format that is better chunked for computation speed to be directly loaded. If this was not the case and `array` was converted to `snp_array` using `make_snp_array`, `make_snp_array` would not pre-compute `snp_array` but instead create the 'instructions' for `snp_array` to be computed in sections when needed. Therefore, instead of computing `snp_array` in sections on demand from the original location, which is slower. `snp_array` can be computed and saved in sections to a new location. This will result in better performance for large arrays as no intermediate computing steps must take place. Instead, it just has to be read from memory similar to how `array` would be because it is so large.
2. In addition, this prevents the need to recreate the MASK of `snp_array` using the function `make_snp_array` (which is precomputed and can be quite costly).

In [56]:
loc = 'testing_array' # Will save the array in a folder called 'testing_array'

In [57]:
save_array(snp_array, file=loc)

We can then reload in the array (in sections on demand).

In [58]:
snp_loaded_array = load_array_from_disk('testing_array/')

In [59]:
%timeit snp_loaded_array.dot(y).compute()

681 ms ± 7.11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


And persist it (to fully load it into memory) to see if we can gain any speed bonus

In [60]:
snp_loaded_array_persist = snp_loaded_array.persist()

In [61]:
%timeit snp_loaded_array_persist.dot(y).compute()

376 ms ± 8.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
